## testing code to assemble files & metadata for STITCH genotyping

In [40]:
import pandas as pd
import numpy as np
import re
import os

### inputs

In [41]:
# genotyping log
geno_log = pd.read_csv('/projects/ps-palmer/hs_rats/round10_1/results/hs_rats_stitch_n16342_20230809_genotype_log.csv')
# flowcells
old_metadata = '/projects/ps-palmer/hs_rats/round10_2/inputs/round10_2_prev_flowcells_metadata'
new_metadata = '/projects/ps-palmer/hs_rats/round10_2/inputs/round10_2_new_flowcells_metadata'
# previous round's metadata
prev_metadata = pd.read_csv('/projects/ps-palmer/hs_rats/round10_1/round10_1_metadata.csv')
# analysis directory
reference_genome = '/projects/ps-palmer/reference_genomes/rat/mRatBN7.2.fa'
ref_gen_prefix = reference_genome[:reference_genome.rfind('.')]
ref_gen = ref_gen_prefix[ref_gen_prefix.rfind('/') + 1: ]
ref_gen

'mRatBN7.2'

### set up previous metadata

In [124]:
# get passing samples from previous round
good_sequences = ['analysis', 'keep']
use_rfids = geno_log[geno_log['sample_use'].isin(good_sequences)]['sample_id']
print(geno_log.shape)
print(len(use_rfids))

(16342, 33)
15966


In [125]:
prev_metadata.columns

Index(['rfid', 'barcode', 'pcr_barcode', 'library_name', 'sex', 'coatcolor',
       'project_name', 'organism', 'strain', 'runid', 'fastq_files', 'bam',
       'dams', 'sires'],
      dtype='object')

In [126]:
# add sample id to previous metadata
prev_metadata['sample_id'] = prev_metadata['library_name']+ '_' + prev_metadata['rfid']
md_cols = ['sample_id', 'rfid', 'library_name', 'project_name', 'barcode', 'pcr_barcode', 'runid',  'sex', 
           'coatcolor', 'dams', 'sires', 'organism', 'strain', 'fastq_files', 'bam']
prev_metadata = prev_metadata[md_cols]
prev_metadata.head()

,sample_id,rfid,library_name,project_name,barcode,pcr_barcode,runid,sex,coatcolor,dams,sires,organism,strain,fastq_files,bam
0,Riptide01_933000120117306,933000120117306,Riptide01,u01_olivier_george_cocaine,TGCACTAG,8.0,200221_A00953_0069_BH5T5LDSXY,F,BROWN,71838_3,71823_1,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
1,Riptide01_933000120117307,933000120117307,Riptide01,u01_olivier_george_cocaine,CGTAGCAT,8.0,200221_A00953_0069_BH5T5LDSXY,M,BLACKHOOD,71836_3,71820_2,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
2,Riptide01_933000120117308,933000120117308,Riptide01,u01_olivier_george_cocaine,TACGAACC,8.0,200221_A00953_0069_BH5T5LDSXY,M,BROWN,71804_3,71822_2,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
3,Riptide01_933000120117311,933000120117311,Riptide01,u01_olivier_george_cocaine,CACTAGAC,8.0,200221_A00953_0069_BH5T5LDSXY,F,BLACKHOOD,71799_3,71836_2,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
4,Riptide01_933000120117312,933000120117312,Riptide01,u01_olivier_george_cocaine,TGGTCATG,8.0,200221_A00953_0069_BH5T5LDSXY,F,BROWNHOOD,71838_3,71823_1,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...


In [127]:
# keep only previous samples that passed QC
prev_use = prev_metadata[prev_metadata['sample_id'].isin(use_rfids)]
prev_use.shape

(15966, 15)

In [ ]:
# # list of metadata columns to keep/concatenate
# md_columns = ['Sample_ID', 'Sample_Name', 'Library_ID', 'Sample_Project', 'Sample_Barcode', 'Fastq_Files',
#               'rfid', 'library_name', 'project_name', 'flowcell_id', 'runid', 'barcode', 'pcr_barcode', 
#               'organism', 'strain', 'sex', 'coatcolor', 'sires', 'dams', 'fastq_files', 'bam']

# with open(old_metadata, 'r') as file:
#     md_paths = file.readlines()

# # emtpy list for old metadata
# old_md = []

# for path in md_paths:

#     path = path.strip() 
#     df = pd.read_csv(path)
    
#     # check and insert missing columns with NaN values
#     for col in md_columns:
#         if col not in df.columns:
#             df[col] = pd.Series([None] * len(df), name=col)

#     # reorder columns to allow concatenation
#     df = df[md_columns]

#     old_md.append(df)

# # concatenate all metradata into a single dataframe
# all_old_md = pd.concat(old_md, ignore_index=True)

# all_old_md.shape

(10078, 21)

In [128]:
# function to find the bam file associated with a sample ID
def find_bam_file(sample_name):
    for bam in bamfiles:
            if sample_name in bam:
                bam_out = f'{bam_dir}/{bam}'
                return bam_out
                break

# list of metadata columns to keep/concatenate
md_columns = ['Sample_ID', 'rfid', 'library_name', 'project_name', 'barcode', 'pcr_barcode',
              'runid', 'sex', 'coatcolor', 'dams', 'sires', 'organism', 'strain', 'fastq_files', 'bam']

with open(new_metadata, 'r') as file:
    md_paths = file.readlines()

# emtpy list for old metadata
new_md = []

for path in md_paths:

    md_path = path.strip() 
    df = pd.read_csv(md_path)
    flowcell_dir = md_path.split('/demux')[0]
    bam_dir = f'{flowcell_dir}/{ref_gen}/bams'
    
    # check and insert missing columns with NaN values
    for col in md_columns:
        if col not in df.columns:
            df[col] = pd.Series([None] * len(df), name=col)

    # reorder columns to allow concatenation
    df = df[md_columns]
    df.rename({'Sample_ID':'sample_id'},axis=1,inplace=True)

     # list all bam files in the flowcell directory
    bamfiles = [file for file in os.listdir(bam_dir) if file.endswith('.bam')]

    # add bam files to metadata
    df['bam'] = df['sample_id'].apply(find_bam_file)

    new_md.append(df)

# concatenate all new metradata into a single dataframe
all_new_md = pd.concat(new_md, ignore_index=True)


In [129]:
print(all_new_md.shape)
print(all_new_md.columns)
all_new_md.head()

(2805, 15)
Index(['sample_id', 'rfid', 'library_name', 'project_name', 'barcode',
       'pcr_barcode', 'runid', 'sex', 'coatcolor', 'dams', 'sires', 'organism',
       'strain', 'fastq_files', 'bam'],
      dtype='object')


,sample_id,rfid,library_name,project_name,barcode,pcr_barcode,runid,sex,coatcolor,dams,sires,organism,strain,fastq_files,bam
0,Riptide211_119110_1M,119110_1M,Riptide211,retired_breeders_mcw,CTCTCACA,1.0,None,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,Riptide211_S1_L004_R1_001.fastq.gz; Riptide211...,/projects/ps-palmer/hs_rats/230712_A01535_0357...
1,Riptide211_119111_1M,119111_1M,Riptide211,retired_breeders_mcw,TGACTGAC,1.0,None,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,Riptide211_S1_L004_R1_001.fastq.gz; Riptide211...,/projects/ps-palmer/hs_rats/230712_A01535_0357...
2,Riptide211_933000320188330,933000320188330,Riptide211,u01_peter_kalivas_italy,GAAGCTTC,1.0,None,F,BROWNHOOD,76857_2,76910_1,rat,Heterogeneous stock,Riptide211_S1_L004_R1_001.fastq.gz; Riptide211...,/projects/ps-palmer/hs_rats/230712_A01535_0357...
3,Riptide211_933000320188332,933000320188332,Riptide211,u01_peter_kalivas_italy,GATGCATC,1.0,None,M,BROWNHOOD,76857_2,76910_1,rat,Heterogeneous stock,Riptide211_S1_L004_R1_001.fastq.gz; Riptide211...,/projects/ps-palmer/hs_rats/230712_A01535_0357...
4,Riptide211_933000320188333,933000320188333,Riptide211,u01_peter_kalivas_italy,CAGTCAGT,1.0,None,M,BLACK,76790_2,76795_1,rat,Heterogeneous stock,Riptide211_S1_L004_R1_001.fastq.gz; Riptide211...,/projects/ps-palmer/hs_rats/230712_A01535_0357...


In [130]:
# concatenate all metadata
all_md = pd.concat([prev_use, all_new_md], ignore_index=True)
all_md['pcr_barcode'] = all_md['pcr_barcode'].fillna(0).astype(int)
df['pcr_barcode'] = df['pcr_barcode'].replace(0, np.nan)
print(all_md.shape)
all_md.head()

(18771, 15)


,sample_id,rfid,library_name,project_name,barcode,pcr_barcode,runid,sex,coatcolor,dams,sires,organism,strain,fastq_files,bam
0,Riptide01_933000120117306,933000120117306,Riptide01,u01_olivier_george_cocaine,TGCACTAG,8,200221_A00953_0069_BH5T5LDSXY,F,BROWN,71838_3,71823_1,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
1,Riptide01_933000120117307,933000120117307,Riptide01,u01_olivier_george_cocaine,CGTAGCAT,8,200221_A00953_0069_BH5T5LDSXY,M,BLACKHOOD,71836_3,71820_2,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
2,Riptide01_933000120117308,933000120117308,Riptide01,u01_olivier_george_cocaine,TACGAACC,8,200221_A00953_0069_BH5T5LDSXY,M,BROWN,71804_3,71822_2,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
3,Riptide01_933000120117311,933000120117311,Riptide01,u01_olivier_george_cocaine,CACTAGAC,8,200221_A00953_0069_BH5T5LDSXY,F,BLACKHOOD,71799_3,71836_2,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...
4,Riptide01_933000120117312,933000120117312,Riptide01,u01_olivier_george_cocaine,TGGTCATG,8,200221_A00953_0069_BH5T5LDSXY,F,BROWNHOOD,71838_3,71823_1,rat,Heterogeneous stock,Hbim_01_Plate_08_S8_L003_R1_001.fastq.gz;Hbim_...,/projects/ps-palmer/hs_rats/200221_A00953_0069...


In [131]:
all_md.tail()

,sample_id,rfid,library_name,project_name,barcode,pcr_barcode,runid,sex,coatcolor,dams,sires,organism,strain,fastq_files,bam
18766,p50-leah-solberg-woods-rnaseq_62333-4-20,62333-4-20,p50-leah-solberg-woods-rnaseq,p50_leah_solberg_woods_rnaseq,NaN,0,NaN,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,NaN,/projects/ps-palmer/hs_rats/p50_leah_solberg_w...
18767,p50-leah-solberg-woods-rnaseq_62383-4-21,62383-4-21,p50-leah-solberg-woods-rnaseq,p50_leah_solberg_woods_rnaseq,NaN,0,NaN,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,NaN,/projects/ps-palmer/hs_rats/p50_leah_solberg_w...
18768,p50-leah-solberg-woods-rnaseq_62383-4-81,62383-4-81,p50-leah-solberg-woods-rnaseq,p50_leah_solberg_woods_rnaseq,NaN,0,NaN,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,NaN,/projects/ps-palmer/hs_rats/p50_leah_solberg_w...
18769,p50-leah-solberg-woods-rnaseq_62675-3-22,62675-3-22,p50-leah-solberg-woods-rnaseq,p50_leah_solberg_woods_rnaseq,NaN,0,NaN,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,NaN,/projects/ps-palmer/hs_rats/p50_leah_solberg_w...
18770,p50-leah-solberg-woods-rnaseq_62675-3-26,62675-3-26,p50-leah-solberg-woods-rnaseq,p50_leah_solberg_woods_rnaseq,NaN,0,NaN,NaN,NaN,NaN,NaN,rat,Heterogeneous stock,NaN,/projects/ps-palmer/hs_rats/p50_leah_solberg_w...


In [132]:
all_md.to_csv('/projects/ps-palmer/hs_rats/round10_2/inputs/round10_2_metadata.csv', index=False)

In [37]:
# Read the sample IDs and file paths from the two text files
idfile = '/projects/ps-palmer/hs_rats/round10_2/inputs/round10_2_sample_ids'
bamlist = '/projects/ps-palmer/hs_rats/round10_2/inputs/round10_2_bamlist'

with open(idfile, "r") as sample_file, open(bamlist, "r") as path_file:
    sample_ids = [line.strip() for line in sample_file]
    bam_files = [line.strip() for line in path_file]


In [38]:
print(sample_ids[18742])
print(bam_files[18742])

p50-leah-solberg-woods-rnaseq_62771-5
/projects/ps-palmer/hs_rats/p50_leah_solberg_woods_rnaseq/mRatBN7.2/bams/p50-leah-solberg-woods-rnaseq_62771-5_sorted_mkDup.bam


In [39]:
# Check if the number of sample IDs and file paths match
if len(sample_ids) != len(bam_files):
    print("Mismatch: The number of sample IDs and file paths is different.")
else:
    # Check if each sample ID corresponds to the expected file path
    for i, (sample_id, file_path) in enumerate(zip(sample_ids, bam_files)):
        # print(f'{i}: {sample_id}, {file_path}')
        if sample_id not in file_path:
            print(f"Mismatch on line {i+1}: Sample ID {sample_id} does not correspond to the bam file {file_path}")
            break
    else:
        print("All samples/files correspond.")


All samples/files correspond.


In [11]:
sample_ids[1] not in bam_files[1]

False

In [24]:
if sample_ids[18742] not in bam_files[18742]:
    print('mismatched')

mismatched
